In [1]:
# The following notebook will get two different datasets of explanatory variables: temporal an non-temporal
# related. 
# In order to improve the speed time, this notebook will create the respective datasets and it will send a task to 
# EarthEngine with a ReduceByRegion operation, we have proved that this method is faster than using the individual
# calls to the API.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import ee
from gee_scripts.get_sources import get_s1_image, get_gldas, get_gpm, get_hansen
from gee_scripts.get_sources import get_srtm, get_globcover, get_gedi, get_gldas_stats
ee.Initialize(project="ee-marortpab")

The objective would be to loop over the points or the dates...<br>
After testing this script https://code.earthengine.google.com/b18e876cca44266be704924b7354ddff <br>
I found out that the best way to do it is to loop over the dates, and then pass the reduceregions. <br>


# 1. Read data

In [4]:
df = pd.read_csv('data/field_data_unique_coords.csv')
df["date"] = pd.to_datetime(df["date"])
len(df)
df

,id,date,source,lon,lat,gwl_cm
0,02_AHL_SBG-B076,2020-11-05,pkeg,117.007750,3.937760,-37.0
1,02_AHL_SBG-B076,2020-11-17,pkeg,117.007750,3.937760,-39.0
2,02_AHL_SBG-B076,2020-12-05,pkeg,117.007750,3.937760,-39.0
3,02_AHL_SBG-B076,2020-12-16,pkeg,117.007750,3.937760,-35.0
4,02_AHL_SBG-B076,2021-01-02,pkeg,117.007750,3.937760,-34.0
...,...,...,...,...,...,...
267213,kecil1,2019-10-26,old_brg,113.805611,-2.856089,-302.1
267214,kecil1,2019-10-27,old_brg,113.805611,-2.856089,-302.3
267215,kecil1,2019-10-31,old_brg,113.805611,-2.856089,-302.3
267216,kecil1,2019-11-02,old_brg,113.805611,-2.856089,-302.3


##########################
## Set type of output
##########################

In [5]:
# This notebook can be run entirely, if we want to task the orders to GEE we'll set this variable to True
send_task = False

##########################

In [6]:
unique_coords = df[["id", "lon", "lat"]].drop_duplicates()
unique_coords.head()
len(unique_coords)

# Convert them as a geodataframe and save them
geometry = [Point(xy) for xy in zip(unique_coords.lon, unique_coords.lat)]
gdf = gpd.GeoDataFrame(unique_coords, geometry=geometry)

In [7]:
# Read regions shapefile

# I have two regions, first is to only the high correlated stations and the second is all the stations
# I will use either depending on the dataset we have selected above

shp_path = Path("data/0_shp/")
region_path = "regions_to_request_explanatory_all.gpkg"

gdf_regions = gpd.GeoDataFrame.from_file(shp_path/region_path)
gdf_regions

,region_id,geometry
0,1,"MULTIPOLYGON (((96.37854 4.01317, 96.76923 3.9..."
1,2,"MULTIPOLYGON (((102.96446 -0.63790, 104.82488 ..."
2,3,"MULTIPOLYGON (((140.00836 -7.80760, 140.75163 ..."
3,4,"MULTIPOLYGON (((105.23245 -2.56075, 105.62785 ..."
4,5,"MULTIPOLYGON (((101.59551 1.61281, 101.45686 0..."
5,6,"MULTIPOLYGON (((100.69365 2.01094, 100.81080 2..."
6,7,"MULTIPOLYGON (((108.80424 1.60848, 109.83126 1..."
7,8,"MULTIPOLYGON (((110.98152 -2.86934, 114.00610 ..."
8,9,"MULTIPOLYGON (((132.99060 -0.68691, 133.43736 ..."
9,10,"MULTIPOLYGON (((116.84967 3.98347, 117.30926 3..."


In [8]:
# Remove those date where the gwl measure is out of reasonable range
upper_thres = 20
lower_thres = -100

# not apllying this right now.... df = df[(df.gwl_cm < upper_thres) & (df.gwl_cm > lower_thres)]

# Get the coordinates of the individual points

unique_coords = df[["id", 'lon', 'lat']].drop_duplicates()
len(df), len(unique_coords)

(267218, 2074)

In [9]:
# Create geodataframe from x y coordinates
gdf_unique_coords = gpd.GeoDataFrame(unique_coords, geometry=gpd.points_from_xy(unique_coords.lon, unique_coords.lat), crs="EPSG:4326")

# Add the region id to each point
gdf_unique_coords = gpd.sjoin(gdf_unique_coords, gdf_regions[["region_id", "geometry"]], how="left", predicate="within")

In [10]:
# Failing datasets array(['BRG_150710_01', 'BRG_621107_03'], dtype=object)
gdf_unique_coords[gdf_unique_coords.id.isin(["BRG_621107_03", "BRG_150710_01"])]

,id,lon,lat,geometry,index_right,region_id
222645,BRG_150710_01,103.900168,-1.274317,POINT (103.90017 -1.27432),1,2
245948,BRG_621107_03,114.220600,-2.654022,POINT (114.22060 -2.65402),7,8


In [11]:
gdf_unique_coords

,id,lon,lat,geometry,index_right,region_id
0,02_AHL_SBG-B076,117.007750,3.937760,POINT (117.00775 3.93776),9,10
65,02_AHL_SBG-B101,117.010120,3.931860,POINT (117.01012 3.93186),9,10
130,02_AHL_SBG-B103,117.005210,3.926090,POINT (117.00521 3.92609),9,10
195,02_AHL_SBG-C003,117.145430,3.903400,POINT (117.14543 3.90340),9,10
260,02_AHL_SBG-C006,117.148320,3.919380,POINT (117.14832 3.91938),9,10
...,...,...,...,...,...,...
266127,ij2,114.022576,-2.573375,POINT (114.02258 -2.57337),7,8
266206,jambi1,103.589975,-1.238478,POINT (103.58997 -1.23848),1,2
266449,kalbar1,109.394853,-0.210225,POINT (109.39485 -0.21022),6,7
266842,kalteng1,114.058131,-2.319728,POINT (114.05813 -2.31973),7,8


# 1. Get temporal explanatory variables

In [12]:
gdf_regions

,region_id,geometry
0,1,"MULTIPOLYGON (((96.37854 4.01317, 96.76923 3.9..."
1,2,"MULTIPOLYGON (((102.96446 -0.63790, 104.82488 ..."
2,3,"MULTIPOLYGON (((140.00836 -7.80760, 140.75163 ..."
3,4,"MULTIPOLYGON (((105.23245 -2.56075, 105.62785 ..."
4,5,"MULTIPOLYGON (((101.59551 1.61281, 101.45686 0..."
5,6,"MULTIPOLYGON (((100.69365 2.01094, 100.81080 2..."
6,7,"MULTIPOLYGON (((108.80424 1.60848, 109.83126 1..."
7,8,"MULTIPOLYGON (((110.98152 -2.86934, 114.00610 ..."
8,9,"MULTIPOLYGON (((132.99060 -0.68691, 133.43736 ..."
9,10,"MULTIPOLYGON (((116.84967 3.98347, 117.30926 3..."


In [13]:
gdf_unique_coords

,id,lon,lat,geometry,index_right,region_id
0,02_AHL_SBG-B076,117.007750,3.937760,POINT (117.00775 3.93776),9,10
65,02_AHL_SBG-B101,117.010120,3.931860,POINT (117.01012 3.93186),9,10
130,02_AHL_SBG-B103,117.005210,3.926090,POINT (117.00521 3.92609),9,10
195,02_AHL_SBG-C003,117.145430,3.903400,POINT (117.14543 3.90340),9,10
260,02_AHL_SBG-C006,117.148320,3.919380,POINT (117.14832 3.91938),9,10
...,...,...,...,...,...,...
266127,ij2,114.022576,-2.573375,POINT (114.02258 -2.57337),7,8
266206,jambi1,103.589975,-1.238478,POINT (103.58997 -1.23848),1,2
266449,kalbar1,109.394853,-0.210225,POINT (109.39485 -0.21022),6,7
266842,kalteng1,114.058131,-2.319728,POINT (114.05813 -2.31973),7,8


# 2. Get temporal explanatory

In [14]:
base_selectors = ["system:index", "lat", "lon", "id", "date"]
s1_selectors = ["LIA", "VH", "VV", "VVVH_ratio", "angle"]
gldas_selectors = ['sm_1', 'sm_3', 'sm_7', 'sm_30']
gpm_selectors = ['precipitation', 'prec_3', 'prec_7', 'prec_30']

def get_temporal_explanatory(region_id):
    """Get the explanatory temporal based variables"""

    region = gdf_regions[gdf_regions.region_id == region_id].to_crs("EPSG:4326")[:]
    dates = df[df.id.isin(gdf_unique_coords[gdf_unique_coords.region_id == region_id].id.unique())].date.unique()
    points = gdf_unique_coords[gdf_unique_coords.region_id == region_id][["id", "geometry", "lat", "lon"]].to_crs("EPSG:4326")

    # print(len(dates), len(points))
    # Convert to ee elements

    ee_dates = ee.FeatureCollection(ee.List([ ee.Feature(None, {"date": date}) for date in dates]))
    ee_points = ee.FeatureCollection(points.__geo_interface__)
    ee_region = ee.FeatureCollection(region.__geo_interface__)

    def get_sources(date_feature):

        date_range = ee.Date(date_feature.get("date")).getRange('day')

        s1_composite = get_s1_image(date_range, ee_region)

        return s1_composite.set({
         "numberOfBands" : s1_composite.bandNames().size(),
         "date" : ee.Date(date_feature.get("date"))
         })

    def reduce_composite(composite):

        # Filter the extra data with the matching date
        date = composite.get("date")
        date_range = ee.Date(date).getRange('day')

        gldas_composite = get_gldas(date_range, ee_region)
        gpm_composite = get_gpm(date_range, ee_region)

        composite = (ee.Image(composite)
            .addBands(gldas_composite)
            .addBands(gpm_composite)
        )

        return composite.reduceRegions(**{
         "collection" : ee_points,
         "reducer" : ee.Reducer.first(),
         "scale" : 10,
         "tileScale" : 16
        }).filter(ee.Filter.notNull(['VH'])).map(lambda feature: feature.set({
         "date" : date
        }))


    task = (ee_dates
         .map(get_sources)
         .filter(ee.Filter.gt('numberOfBands', 0))
         .map(reduce_composite).flatten()
    )

    task_name = f"All_temporal_non_resample_at_all_region_{region_id}_dates_{len(dates)}_points_{len(points)}_with_date_lon_lat"

    ee_task = ee.batch.Export.table.toDrive(**{
      "collection": task, 
      "folder" : "INDONESIA_GWL",
      "description": task_name,
      "selectors": base_selectors + s1_selectors + gldas_selectors + gpm_selectors
    })

    # Uncoment to start the task
    not send_task or ee_task.start()
    print("Exported" if send_task else "Not exported", task_name)

[get_temporal_explanatory(region_id) for region_id in gdf_regions.region_id.unique()]

Not exported All_temporal_non_resample_at_all_region_1_dates_520_points_24_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_2_dates_1785_points_148_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_3_dates_479_points_1_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_4_dates_988_points_348_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_5_dates_1799_points_718_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_6_dates_489_points_43_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_7_dates_1274_points_477_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_8_dates_1673_points_221_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_9_dates_379_points_17_with_date_lon_lat
Not exported All_temporal_non_resample_at_all_region_10_dates_846_points_77_with_date_lon_lat


[None, None, None, None, None, None, None, None, None, None]

# 2.2 Get "yearly" temporal explanatory variables (Hansen)

In [15]:
hansen_selectors = ["year", "B3","B4","B5","B7","ndvi","ndmi","ndbri"]

# get all the years from the field data
years = sorted([y for y in df.date.dt.year.unique() if y != 2013] )

for year in years:

    points = df[["id", "lon", "lat"]].drop_duplicates()
    points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat), crs="EPSG:4326")
    ee_points = ee.FeatureCollection(points.__geo_interface__)

    image = get_hansen(year)

    result = image.reduceRegions(**{
        "collection" : ee_points,
        "reducer" : ee.Reducer.first(),
        "scale" : 30,
        "tileScale" : 16
    }).map(lambda feature: feature.set("year", str(year)))
    
    task_name = f"Hansen_year_{year}_points_{len(points)}_f"

    ee_task = ee.batch.Export.table.toDrive(**{
        "collection": result, 
        "folder" : "INDONESIA_GWL",
        "description": f"Hansen_year_{year}_points_{len(points)}_f",
        "selectors": base_selectors + hansen_selectors
    })

    not send_task or ee_task.start()
    print("Exported" if send_task else "Not exported", task_name)


Not exported Hansen_year_2018_points_2074_f
Not exported Hansen_year_2019_points_2074_f
Not exported Hansen_year_2020_points_2074_f
Not exported Hansen_year_2021_points_2074_f
Not exported Hansen_year_2022_points_2074_f
Not exported Hansen_year_2023_points_2074_f


## 3. Get non temporal explanatory variables (others)

In [16]:
# This dataset is not too computational expensive, so we are not forced to chunk it
# We'll try to get all the points at once, not by region (so we won't filter by region)
region = gdf_regions.to_crs("EPSG:4326")[:]
ee_region = ee.FeatureCollection(region.__geo_interface__)
points = gdf_unique_coords[["id", "geometry", "lat", "lon"]].rename(columns={"id": "id"}).to_crs("EPSG:4326")
ee_points = ee.FeatureCollection(points.__geo_interface__)
len(points)

2074

In [17]:
composite = (
    get_srtm()
        .addBands(get_globcover())
        .addBands(get_gedi(ee_region))
        .addBands(get_gldas_stats(ee_region))
)
composite.bandNames().getInfo()

result = composite.reduceRegions(**{
    "collection" : ee_points,
    "reducer" : ee.Reducer.first(),
    "scale" : 10,
    "tileScale" : 16
}).filter(ee.Filter.notNull(['canopy_height']))

task_name = f"All_Non_temporal_points_{len(points)}"

ee_task = ee.batch.Export.table.toDrive(**{
    "collection": result, 
    "folder" : "INDONESIA_GWL",
    "description":task_name,
    "selectors": base_selectors + ['elevation', 'aspect', 'slope', 'land_cov', 'canopy_height', "gldas_mean", "gldas_stddev"]
})

# Uncoment to start the task
not send_task or ee_task.start()
print("Exported" if send_task else "Not exported", task_name)

Not exported All_Non_temporal_points_2074


## 4. Get Extra Non temporal explanatory variables (others)

This data comes from https://code.earthengine.google.com/6c3eeb929a5ee8a42f55234b58796c0a


In [18]:
phu = ee.FeatureCollection("users/marortpab/FAO/SEPAL/2023_trainings/smm/AOI__Province__865_PHUs__INDONESIA")
water = ee.FeatureCollection("users/marortpab/FAO/SEPAL/2023_trainings/smm/water_bodies_phu_buff_1_km_def")
canals = ee.FeatureCollection("users/marortpab/FAO/SEPAL/2023_trainings/smm/prims_canal_data")

hydro_dir = ee.Image('WWF/HydroSHEDS/03DIR')
drain_direction = hydro_dir.select('b1').rename('dir')
hydro_acc = ee.Image('WWF/HydroSHEDS/15ACC')
flow_acc = hydro_acc.select('b1').rename('acc');


In [19]:
# //Merge water bodies and canals

all_water = water.merge(canals)
distance = all_water.distance(25000)

composite = (
    distance
        .addBands(drain_direction)
        .addBands(flow_acc)
)
composite.bandNames().getInfo()


['distance', 'dir', 'acc']

In [20]:
result = composite.reduceRegions(**{
    "collection" : ee_points,
    "reducer" : ee.Reducer.first(),
    "scale" : 10,
    "tileScale" : 16
}).filter(ee.Filter.notNull(['distance']))

task_name = f"All_Non_temporal_extra_points_latlon_{len(points)}"

ee_task = ee.batch.Export.table.toDrive(**{
    "collection": result, 
    "folder" : "INDONESIA_GWL",
    "description":task_name,
    "selectors": base_selectors + ['distance', 'dir', 'acc']
})

# Uncoment to start the task
not send_task or ee_task.start()
print("Exported" if send_task else "Not exported", task_name)

Not exported All_Non_temporal_extra_points_latlon_2074


# 4. Merge explanatory variables

## 4.1 Read temporal variables

In [21]:
from pathlib import Path
import pandas as pd

In [22]:
explanatory_path = Path("data/7_training_data/")
dataset = "all"
temporal_file_names_groups = {
    "all" : [
        "All_temporal_non_resample_at_all_region_1_dates_520_points_24_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_2_dates_1773_points_148_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_3_dates_479_points_1_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_4_dates_988_points_348_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_5_dates_1796_points_717_with_date.csv",
        "All_temporal_non_resample_at_all_region_6_dates_489_points_43_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_7_dates_1274_points_477_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_8_dates_1671_points_220_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_9_dates_379_points_17_with_date_lon_lat.csv",
        "All_temporal_non_resample_at_all_region_10_dates_846_points_77_with_date_lon_lat.csv",
    ]
}

In [23]:
# Dataset is the name of the type of data we're using (high_corr or all) (it's assigned at the beginning of the notebook)
temporal_file_names = temporal_file_names_groups[dataset]

# get and concatenate all the dataframes
temp_explanatory_dfs = pd.concat([
            pd.read_csv(explanatory_path/file_name) 
            for file_name 
            in temporal_file_names
        ], 
)

temp_explanatory_dfs["date"] = pd.to_datetime(temp_explanatory_dfs["date"])
temp_explanatory_dfs

,system:index,lat,lon,id,date,LIA,VH,VV,VVVH_ratio,angle,sm_1,sm_3,sm_7,sm_30,precipitation,prec_3,prec_7,prec_30
0,13_40018,3.809687,96.451939,121_APC_A21,2021-11-18,41.518192,-12.389068,-4.963172,0.261232,36.802202,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
1,13_40056,3.811906,96.468786,121_APC_B09,2021-11-18,37.709442,-11.907431,-5.864731,0.194680,35.960575,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
2,13_40094,3.806685,96.459393,121_APC_B14,2021-11-18,35.779500,-12.803118,-5.593195,0.223412,35.960575,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
3,13_40132,3.855798,96.523820,121_APC_D11,2021-11-18,36.007309,-13.151474,-5.039535,0.264961,36.007309,38.687000,38.470375,37.954553,38.176337,0.480000,0.242308,0.110308,0.418293
4,13_40170,3.850565,96.515498,121_APC_D15,2021-11-18,37.853025,-14.973237,-4.171970,0.350833,36.007309,38.687000,38.470375,37.954553,38.176337,0.480000,0.242308,0.110308,0.418293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,840_5023,3.562450,117.137710,02_AHL_SSP-F153,2023-01-22,35.852239,-16.009284,-7.527672,0.151633,35.958145,42.348999,37.115084,37.660375,38.239854,0.022500,0.007826,0.448148,0.610195
5191,840_5088,3.542370,117.130950,02_AHL_SSP-F173,2023-01-22,35.036420,-12.622485,-7.525078,0.122134,35.958125,42.348999,37.115084,37.660375,38.239854,0.022500,0.007826,0.448148,0.610195
5192,840_5153,3.598300,117.201030,02_AHL_SSP-G032,2023-01-22,39.640818,-13.096971,-8.278228,0.099642,35.132320,42.348999,37.115084,37.660375,38.239854,0.030000,0.013750,0.303273,0.510766
5193,840_5218,3.601620,117.193570,02_AHL_SSP-G044,2023-01-22,35.764655,-11.508965,-8.861900,0.059312,35.132320,42.348999,37.115084,37.660375,38.239854,0.012500,0.004348,0.052407,0.567561


In [24]:
# Dataset is the name of the type of data we're using (high_corr or all) (it's assigned at the beginning of the notebook)
temporal_file_names = temporal_file_names_groups[dataset]

# get and concatenate all the dataframes
temp_explanatory_dfs = pd.concat([
            pd.read_csv(explanatory_path/file_name) 
            for file_name 
            in temporal_file_names
        ], 
)

temp_explanatory_dfs["date"] = pd.to_datetime(temp_explanatory_dfs["date"])
temp_explanatory_dfs

,system:index,lat,lon,id,date,LIA,VH,VV,VVVH_ratio,angle,sm_1,sm_3,sm_7,sm_30,precipitation,prec_3,prec_7,prec_30
0,13_40018,3.809687,96.451939,121_APC_A21,2021-11-18,41.518192,-12.389068,-4.963172,0.261232,36.802202,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
1,13_40056,3.811906,96.468786,121_APC_B09,2021-11-18,37.709442,-11.907431,-5.864731,0.194680,35.960575,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
2,13_40094,3.806685,96.459393,121_APC_B14,2021-11-18,35.779500,-12.803118,-5.593195,0.223412,35.960575,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
3,13_40132,3.855798,96.523820,121_APC_D11,2021-11-18,36.007309,-13.151474,-5.039535,0.264961,36.007309,38.687000,38.470375,37.954553,38.176337,0.480000,0.242308,0.110308,0.418293
4,13_40170,3.850565,96.515498,121_APC_D15,2021-11-18,37.853025,-14.973237,-4.171970,0.350833,36.007309,38.687000,38.470375,37.954553,38.176337,0.480000,0.242308,0.110308,0.418293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,840_5023,3.562450,117.137710,02_AHL_SSP-F153,2023-01-22,35.852239,-16.009284,-7.527672,0.151633,35.958145,42.348999,37.115084,37.660375,38.239854,0.022500,0.007826,0.448148,0.610195
5191,840_5088,3.542370,117.130950,02_AHL_SSP-F173,2023-01-22,35.036420,-12.622485,-7.525078,0.122134,35.958125,42.348999,37.115084,37.660375,38.239854,0.022500,0.007826,0.448148,0.610195
5192,840_5153,3.598300,117.201030,02_AHL_SSP-G032,2023-01-22,39.640818,-13.096971,-8.278228,0.099642,35.132320,42.348999,37.115084,37.660375,38.239854,0.030000,0.013750,0.303273,0.510766
5193,840_5218,3.601620,117.193570,02_AHL_SSP-G044,2023-01-22,35.764655,-11.508965,-8.861900,0.059312,35.132320,42.348999,37.115084,37.660375,38.239854,0.012500,0.004348,0.052407,0.567561


In [25]:
# drop duplicate id and date
temp_explanatory_dfs.drop_duplicates(["id", "date"])

,system:index,lat,lon,id,date,LIA,VH,VV,VVVH_ratio,angle,sm_1,sm_3,sm_7,sm_30,precipitation,prec_3,prec_7,prec_30
0,13_40018,3.809687,96.451939,121_APC_A21,2021-11-18,41.518192,-12.389068,-4.963172,0.261232,36.802202,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
1,13_40056,3.811906,96.468786,121_APC_B09,2021-11-18,37.709442,-11.907431,-5.864731,0.194680,35.960575,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
2,13_40094,3.806685,96.459393,121_APC_B14,2021-11-18,35.779500,-12.803118,-5.593195,0.223412,35.960575,38.418999,38.516500,37.875393,38.052337,0.567273,0.296923,0.136061,0.480418
3,13_40132,3.855798,96.523820,121_APC_D11,2021-11-18,36.007309,-13.151474,-5.039535,0.264961,36.007309,38.687000,38.470375,37.954553,38.176337,0.480000,0.242308,0.110308,0.418293
4,13_40170,3.850565,96.515498,121_APC_D15,2021-11-18,37.853025,-14.973237,-4.171970,0.350833,36.007309,38.687000,38.470375,37.954553,38.176337,0.480000,0.242308,0.110308,0.418293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,840_5023,3.562450,117.137710,02_AHL_SSP-F153,2023-01-22,35.852239,-16.009284,-7.527672,0.151633,35.958145,42.348999,37.115084,37.660375,38.239854,0.022500,0.007826,0.448148,0.610195
5191,840_5088,3.542370,117.130950,02_AHL_SSP-F173,2023-01-22,35.036420,-12.622485,-7.525078,0.122134,35.958125,42.348999,37.115084,37.660375,38.239854,0.022500,0.007826,0.448148,0.610195
5192,840_5153,3.598300,117.201030,02_AHL_SSP-G032,2023-01-22,39.640818,-13.096971,-8.278228,0.099642,35.132320,42.348999,37.115084,37.660375,38.239854,0.030000,0.013750,0.303273,0.510766
5193,840_5218,3.601620,117.193570,02_AHL_SSP-G044,2023-01-22,35.764655,-11.508965,-8.861900,0.059312,35.132320,42.348999,37.115084,37.660375,38.239854,0.012500,0.004348,0.052407,0.567561


## 4.2 Read Hansen yearly variables

In [26]:
explanatory_path = Path("data/7_training_data/")
hansen_file_names = [
    "Hansen_year_2018_points_2075_f.csv",
    "Hansen_year_2019_points_2075_f.csv",
    "Hansen_year_2020_points_2075_f.csv",
    "Hansen_year_2021_points_2075_f.csv",
    "Hansen_year_2022_points_2075_f.csv",
    "Hansen_year_2023_points_2075_f.csv"
]

hansen_df = pd.concat([
    pd.read_csv(explanatory_path/file_name) 
    for file_name 
    in hansen_file_names
], axis=0)
hansen_df

,system:index,lat,lon,id,date,year,B3,B4,B5,B7,ndvi,ndmi,ndbri
0,0,1.519444,102.099167,BRG_140301_01,NaN,2018,16,91,66,27,0.700935,0.159236,0.542373
1,1797,1.451944,102.181944,BRG_140301_02,NaN,2018,19,91,66,28,0.654545,0.159236,0.529412
2,2376,1.511780,102.158660,BRG_140302_01,NaN,2018,25,73,75,39,0.489796,-0.013514,0.303571
3,4173,1.516389,102.433056,BRG_140302_02,NaN,2018,18,89,74,32,0.663551,0.092025,0.471074
4,5970,0.830883,102.354165,BRG_140802_02,NaN,2018,25,58,73,46,0.397590,-0.114504,0.115385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,273175,-2.573375,114.022576,ij2,NaN,2023,18,80,51,20,0.632653,0.221374,0.600000
2071,273254,-1.238478,103.589975,jambi1,NaN,2023,16,78,64,26,0.659574,0.098592,0.500000
2072,273497,-0.210225,109.394853,kalbar1,NaN,2023,30,87,86,43,0.487179,0.005780,0.338462
2073,273890,-2.319728,114.058131,kalteng1,NaN,2023,16,79,50,19,0.663158,0.224806,0.612245


## 4.3 Read non temporal explanatory

In [27]:
# As the non-temporal variables are the same for all the points, we just need to duplicate 
# their results into each of the dates of the points.
# i.e. 1 point with 10 dates will have the same non-temporal variables for each of the 10 dates.

non_temporal_file_name = "All_Non_temporal_points_2074.csv"
non_temporal_df = pd.read_csv(explanatory_path/non_temporal_file_name)
# drop lat and lon
non_temporal_df = non_temporal_df.drop(columns=["lat", "lon"])
non_temporal_df

,system:index,id,elevation,aspect,slope,land_cov,canopy_height,gldas_mean,gldas_stddev
0,0,BRG_140301_01,12,26.573112,2.073173,40,15,32.506657,1.989591
1,1799,BRG_140301_02,13,270.000000,0.927708,40,6,32.530491,1.982611
2,2530,BRG_140302_01,13,270.000000,1.854979,30,4,32.526196,1.994038
3,4174,BRG_140302_02,13,165.959034,3.818558,40,13,32.566113,1.973969
4,5970,BRG_140802_02,20,239.038901,5.392593,160,0,32.519711,1.973292
...,...,...,...,...,...,...,...,...,...
2069,272775,brg6,7,314.990916,1.311649,40,0,31.976336,2.483077
2070,273254,jambi1,17,-0.000000,1.854334,30,10,31.922771,2.548600
2071,273497,kalbar1,8,243.435103,2.073039,12,3,32.688313,2.437983
2072,273890,kalteng1,8,116.546267,2.074387,40,10,31.701403,3.461483


## 4.4 Read extra non temporal explanatory (accumulation, distance to rivers/canals)

In [28]:
# As the non-temporal variables are the same for all the points, we just need to duplicate 
# their results into each of the dates of the points.
# i.e. 1 point with 10 dates will have the same non-temporal variables for each of the 10 dates.

non_temporal_extra_file_name = "All_Non_temporal_extra_points_latlon_2072.csv"
non_temporal_extra_df = pd.read_csv(explanatory_path/non_temporal_extra_file_name)
# drop lat and lon
non_temporal_extra_df = non_temporal_extra_df.drop(columns=["lat", "lon", "date"])
non_temporal_extra_df

,system:index,id,distance,dir,acc
0,0,02_AHL_SBG-B076,4519.468722,1,1
1,65,02_AHL_SBG-B101,5020.676546,128,36
2,130,02_AHL_SBG-B103,4898.636465,64,2
3,195,02_AHL_SBG-C003,1678.548906,4,1
4,260,02_AHL_SBG-C006,1459.469969,128,1
...,...,...,...,...,...
2067,264584,brg6,78.813661,32,2
2068,264984,brg8,67.183029,8,3
2069,266206,jambi1,139.385394,32,1
2070,266449,kalbar1,94.286577,4,17


## 4.4 Create final explanatory variables dataset

In [29]:
len(temp_explanatory_dfs)

383290

In [30]:
# Merge the non-temporal variables with the temporal ones
explanatory_df = temp_explanatory_dfs.merge(non_temporal_df, on="id")
len(explanatory_df)

383668

In [31]:
# Merge with the extra non-temporal variables
explanatory_df = explanatory_df.merge(non_temporal_extra_df, on="id")
len(explanatory_df)

383564

In [32]:
explanatory_df.head()

,system:index_x,lat,lon,id,date,LIA,VH,VV,VVVH_ratio,angle,...,aspect,slope,land_cov,canopy_height,gldas_mean,gldas_stddev,system:index,distance,dir,acc
0,13_40018,3.809687,96.451939,121_APC_A21,2021-11-18,41.518192,-12.389068,-4.963172,0.261232,36.802202,...,258.714418,4.728601,12,5,36.23185,2.540359,40018,908.136558,16,4
1,26_40018,3.809687,96.451939,121_APC_A21,2022-06-10,41.228146,-12.835561,-5.448382,0.233155,36.512148,...,258.714418,4.728601,12,5,36.23185,2.540359,40018,908.136558,16,4
2,30_40018,3.809687,96.451939,121_APC_A21,2023-02-11,33.772167,-12.627489,-6.298591,0.179892,38.231369,...,258.714418,4.728601,12,5,36.23185,2.540359,40018,908.136558,16,4
3,31_40018,3.809687,96.451939,121_APC_A21,2023-02-23,33.771343,-14.834075,-5.877339,0.225530,38.230583,...,258.714418,4.728601,12,5,36.23185,2.540359,40018,908.136558,16,4
4,48_40018,3.809687,96.451939,121_APC_A21,2021-08-26,41.515096,-12.292229,-4.994940,0.257607,36.799110,...,258.714418,4.728601,12,5,36.23185,2.540359,40018,908.136558,16,4


In [33]:
# Merge hansen data with year and id
explanatory_df["year"] = explanatory_df.date.dt.year
hansen_df["year"] = hansen_df["year"].astype(int)
explanatory_df = explanatory_df.merge(hansen_df[["id"] + hansen_selectors], on=["id", "year"], how="left")

# I get more values here because I have requested Hansen for all the years
explanatory_df

,system:index_x,lat,lon,id,date,LIA,VH,VV,VVVH_ratio,angle,...,dir,acc,year,B3,B4,B5,B7,ndvi,ndmi,ndbri
0,13_40018,3.809687,96.451939,121_APC_A21,2021-11-18,41.518192,-12.389068,-4.963172,0.261232,36.802202,...,16,4,2021,18,93,69,28,0.675676,0.148148,0.537190
1,26_40018,3.809687,96.451939,121_APC_A21,2022-06-10,41.228146,-12.835561,-5.448382,0.233155,36.512148,...,16,4,2022,18,93,69,28,0.675676,0.148148,0.537190
2,30_40018,3.809687,96.451939,121_APC_A21,2023-02-11,33.772167,-12.627489,-6.298591,0.179892,38.231369,...,16,4,2023,18,93,69,28,0.675676,0.148148,0.537190
3,31_40018,3.809687,96.451939,121_APC_A21,2023-02-23,33.771343,-14.834075,-5.877339,0.225530,38.230583,...,16,4,2023,18,93,69,28,0.675676,0.148148,0.537190
4,48_40018,3.809687,96.451939,121_APC_A21,2021-08-26,41.515096,-12.292229,-4.994940,0.257607,36.799110,...,16,4,2021,18,93,69,28,0.675676,0.148148,0.537190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384315,810_5283,3.601800,117.175400,02_AHL_SSP-G047,2023-03-23,31.246828,-13.393182,-10.248247,0.048664,35.960678,...,32,1,2023,28,51,80,43,0.291139,-0.221374,0.085106
384316,811_5283,3.601800,117.175400,02_AHL_SSP-G047,2023-05-22,31.245850,-14.819223,-9.667181,0.074998,35.959713,...,32,1,2023,28,51,80,43,0.291139,-0.221374,0.085106
384317,831_5283,3.601800,117.175400,02_AHL_SSP-G047,2021-10-11,30.421554,-13.218312,-7.836919,0.116892,35.135782,...,32,1,2021,14,77,46,18,0.692308,0.252033,0.621053
384318,833_5283,3.601800,117.175400,02_AHL_SSP-G047,2022-07-14,30.419886,-13.768306,-7.838126,0.122516,35.134108,...,32,1,2022,28,51,80,43,0.291139,-0.221374,0.085106


In [34]:
export_vars = [
    'id', 'date', 'LIA', 'VH', 'VV', 'VVVH_ratio',
    'angle', 'sm_1', 'sm_3', 'sm_7', 'sm_30', 'precipitation', 'prec_3',
    'prec_7', 'prec_30', 'elevation',
    'aspect', 'slope', 'land_cov', 'canopy_height', 'gldas_mean',
    'gldas_stddev', 'B3', 'B4',
    'B5', 'B7', 'ndvi', 'ndmi', 'ndbri',
    'distance', 'dir', 'acc'
]

# 4.5 Final step: Merge explanatory variables with response variable

In [35]:
explanatory_with_response_var = df[["source", "id", "date", "gwl_cm", "lat", "lon"]].merge(explanatory_df[export_vars], on=["id", "date"])

# Add day of the year as a variable
explanatory_with_response_var["doy"] = explanatory_with_response_var.date.dt.dayofyear
explanatory_with_response_var.to_csv("data/7_training_data/explanatory_with_response_var_and_source_extra.csv", index=False)

In [36]:
import pandas as pd
pd.read_csv("data/7_training_data/explanatory_with_response_var_and_source_extra.csv")

,source,id,date,gwl_cm,lat,lon,LIA,VH,VV,VVVH_ratio,...,B4,B5,B7,ndvi,ndmi,ndbri,distance,dir,acc,doy
0,pkeg,02_AHL_SBG-B076,2021-06-01,-41.0,3.937760,117.007750,32.785855,-11.481278,-5.556430,0.207099,...,97,61,25,0.644068,0.227848,0.590164,4519.468722,1,1,152
1,pkeg,02_AHL_SBG-B076,2021-08-24,-38.0,3.937760,117.007750,32.785295,-12.812067,-5.960235,0.201164,...,97,61,25,0.644068,0.227848,0.590164,4519.468722,1,1,236
2,pkeg,02_AHL_SBG-B076,2022-04-09,-22.0,3.937760,117.007750,32.795191,-18.437775,-10.636812,0.072032,...,70,96,52,0.359223,-0.156627,0.147541,4519.468722,1,1,99
3,pkeg,02_AHL_SBG-B076,2023-02-15,-31.0,3.937760,117.007750,32.793740,-13.051827,-9.007584,0.076149,...,70,96,52,0.359223,-0.156627,0.147541,4519.468722,1,1,46
4,pkeg,02_AHL_SBG-B101,2021-06-01,-66.0,3.931860,117.010120,39.267563,-9.778736,-6.648683,0.111111,...,75,57,25,0.562500,0.136364,0.500000,5020.676546,128,36,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36165,old_brg,kalteng1,2020-02-24,-23.9,-2.319728,114.058131,32.478706,-12.878587,-8.271499,0.097345,...,81,53,21,0.620000,0.208955,0.588235,189.692997,128,16,55
36166,old_brg,kalteng1,2020-03-07,-31.1,-2.319728,114.058131,32.478589,-13.078360,-6.593164,0.169898,...,81,53,21,0.620000,0.208955,0.588235,189.692997,128,16,67
36167,old_brg,kalteng1,2020-03-19,-14.7,-2.319728,114.058131,32.479968,-12.563061,-7.793232,0.110794,...,81,53,21,0.620000,0.208955,0.588235,189.692997,128,16,79
36168,old_brg,kalteng1,2020-03-31,-27.1,-2.319728,114.058131,32.480674,-13.179518,-7.523130,0.128794,...,81,53,21,0.620000,0.208955,0.588235,189.692997,128,16,91


In [37]:
# Get the list of variables (column names)
variables = explanatory_with_response_var.columns.tolist()

# Print the list of variables
print("List of variables:")
for var in variables:
    print(var)

List of variables:
source
id
date
gwl_cm
lat
lon
LIA
VH
VV
VVVH_ratio
angle
sm_1
sm_3
sm_7
sm_30
precipitation
prec_3
prec_7
prec_30
elevation
aspect
slope
land_cov
canopy_height
gldas_mean
gldas_stddev
B3
B4
B5
B7
ndvi
ndmi
ndbri
distance
dir
acc
doy
